<a href="https://colab.research.google.com/github/sakshamshil/Machine-Learning-Projects/blob/main/Credit%20Card%20Fraud%20Detection/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Work Flow**

Credit Card Data -> Data Pre Processing -> Data Analysis -> Train Test Split -> Logistic Regression -> Evaluation

In [ ]:
#dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
#loading data into pandas dataframe
cc_data = pd.read_csv('/content/creditcard dataset.csv')

In [ ]:
cc_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


V1 V2 .. are some features about the transactions. Since the specific details about these transactions are sensitive, the features has been converted into numerical values using **Principle Component Analysis Method**

In [ ]:
cc_data.shape

(15936, 31)

In [ ]:
cc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15936 entries, 0 to 15935
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    15936 non-null  int64  
 1   V1      15936 non-null  float64
 2   V2      15936 non-null  float64
 3   V3      15936 non-null  float64
 4   V4      15936 non-null  float64
 5   V5      15936 non-null  float64
 6   V6      15936 non-null  float64
 7   V7      15936 non-null  float64
 8   V8      15936 non-null  float64
 9   V9      15936 non-null  float64
 10  V10     15936 non-null  float64
 11  V11     15936 non-null  float64
 12  V12     15936 non-null  float64
 13  V13     15936 non-null  float64
 14  V14     15936 non-null  float64
 15  V15     15936 non-null  float64
 16  V16     15936 non-null  float64
 17  V17     15936 non-null  float64
 18  V18     15936 non-null  float64
 19  V19     15936 non-null  float64
 20  V20     15936 non-null  float64
 21  V21     15936 non-null  float64
 22

In [ ]:
cc_data.isnull().sum()       #checks for missing values in each column

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [ ]:
cc_data['Class'].value_counts()

0.0    15862
1.0       73
Name: Class, dtype: int64

0 -> Legit Transaction

1 -> Fraudulent Transaction

-> The problem with this dataset is that it is highly unbalanced. The legit transactions cover more than 99% of the dataset. We cannot train our Logistic Regression Model with this dataset because it has very less datapoints for fraud transactions, so it will predict every transaction as a legit transaction.

Therefore we will have to process this data.


In [ ]:
#seperating data for analysis
legit = cc_data[cc_data.Class==0]
fraud = cc_data[cc_data.Class==1]

In [ ]:
print(legit.shape, fraud.shape)

(15862, 31) (73, 31)


In [ ]:
#Statistical measures about the data
legit.Amount.describe()

count    15862.000000
mean        66.280151
std        188.898885
min          0.000000
25%          5.522500
50%         15.950000
75%         53.890000
max       7712.430000
Name: Amount, dtype: float64

In [ ]:
fraud.Amount.describe()

count      73.000000
mean       90.307123
std       271.634360
min         0.000000
25%         1.000000
50%         1.000000
75%        99.990000
max      1809.680000
Name: Amount, dtype: float64

In [ ]:
#compare values for both transactions
cc_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,12104.432165,-0.219072,0.25000,0.862854,0.272641,-0.105868,0.124522,-0.112681,-0.016178,0.879120,...,0.028173,-0.064056,-0.161585,-0.035040,0.013076,0.117023,0.037111,0.011206,0.005907,66.280151
1.0,15559.643836,-7.929807,6.19312,-11.997831,6.555050,-5.474984,-2.480356,-8.354317,3.668478,-3.086988,...,0.671905,0.441100,-0.295856,-0.358994,-0.328925,0.216076,0.209323,0.927187,0.061380,90.307123


Dealing with this unbalanced data

###Under-Sampling

Build a sample dataset containing similar distributions of Legit transactions and Fraudulent transactions (no of fraud transactions - 73)

In [ ]:
legit_sample = legit.sample(n = 73)              #This will randomly sample the legit transactions

In [ ]:
#concatenating legit_sample & fraud_sample
new_dataset = pd.concat([legit_sample, fraud], axis = 0)        #axis = 0 since we will add row-wise

In [ ]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
4567,3889,-1.046495,0.264073,3.521599,1.409309,-0.616022,0.596741,-0.515098,0.454217,1.923712,...,-0.238599,-0.257030,-0.223218,0.364550,0.192796,0.920144,-0.024121,0.058908,15.87,0.0
7198,9438,-0.798530,1.750501,1.170244,2.850068,0.271393,0.193895,0.644048,0.339265,-1.066749,...,-0.006455,-0.044003,-0.235337,-0.046397,0.447226,0.319186,-0.128578,-0.004554,53.89,0.0
2454,2021,-0.423613,0.933065,1.565861,0.745475,0.493257,0.486831,0.552198,0.114503,-0.652367,...,0.016797,0.074377,-0.152878,-0.775166,-0.110700,-0.343030,0.158288,0.126757,29.99,0.0
8040,11002,1.305037,-0.083314,0.501800,0.136335,-0.497857,-0.373569,-0.512803,-0.099690,1.923631,...,-0.131582,-0.156093,-0.108479,-0.449336,0.290146,1.085608,-0.096176,-0.004423,15.95,0.0
10683,18067,1.254074,-0.297152,0.885284,-0.594817,-0.992069,-0.574118,-0.674457,-0.080324,3.165088,...,-0.182581,-0.062974,-0.075074,-0.017646,0.513133,-0.703267,0.062867,0.026625,11.85,0.0


In [ ]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
15566,26961,-23.237920,13.487386,-25.188773,6.261733,-17.345188,-4.534989,-17.100492,15.374630,-3.845567,...,1.769708,-1.691973,-1.045673,0.143386,1.611577,-0.221576,1.481233,0.438125,99.99,1.0
15736,27163,-23.914101,13.765942,-25.733734,6.290918,-17.784824,-4.572498,-17.390764,15.794136,-3.819832,...,1.773399,-1.748426,-1.093104,0.147807,1.663792,-0.221024,1.458076,0.430315,99.99,1.0
15751,27187,-24.590245,14.044567,-26.278701,6.320089,-18.224513,-4.609968,-17.681003,16.213627,-3.794093,...,1.777063,-1.804874,-1.140605,0.152234,1.715997,-0.220471,1.434951,0.422492,99.99,1.0
15781,27219,-25.266355,14.323254,-26.823673,6.349248,-18.664251,-4.647403,-17.971212,16.633103,-3.768351,...,1.780701,-1.861318,-1.188167,0.156667,1.768192,-0.219916,1.411855,0.414656,99.99,1.0
15810,27252,-25.942434,14.601998,-27.368650,6.378395,-19.104033,-4.684806,-18.261393,17.052566,-3.742605,...,1.784316,-1.917759,-1.235787,0.161105,1.820378,-0.219359,1.388786,0.406810,99.99,1.0


In [ ]:
new_dataset['Class'].value_counts()

0.0    73
1.0    73
Name: Class, dtype: int64

In [ ]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,12627.041096,-0.130824,0.19928,0.949028,0.656486,0.040913,0.370561,-0.053739,0.102553,0.762927,...,0.103595,-0.121117,-0.159352,-0.056805,0.087541,0.077986,-0.009785,0.011574,0.02068,87.138767
1.0,15559.643836,-7.929807,6.19312,-11.997831,6.555050,-5.474984,-2.480356,-8.354317,3.668478,-3.086988,...,0.671905,0.441100,-0.295856,-0.358994,-0.328925,0.216076,0.209323,0.927187,0.06138,90.307123


###Splitting data into targets and features

In [ ]:
X = new_dataset.drop(columns = 'Class', axis = 1)
Y = new_dataset['Class']

###Splitting data into training and testing data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split (X, Y, test_size = 0.2, stratify = Y, random_state = 2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(146, 30) (116, 30) (30, 30)


##Building the model

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

###Evaluation using accuracy_score

In [ ]:
#on Training Data
train_predict = model.predict(X_train)
acc_score = accuracy_score(train_predict, Y_train)
print(acc_score)

0.9741379310344828


In [ ]:
#on Testing Data
test_predict = model.predict(X_test)
acc_score = accuracy_score(test_predict, Y_test)
print(acc_score)

0.9333333333333333


##Building a Predictive Model

In [ ]:
#input_data = [12,1.10321543528383,-0.0402962145973447,1.2673320885949,1.28909146962552,-0.735997163604068,0.288069162976262,-0.586056786337461,0.189379713679593,0.782332891785191,-0.267975066537173,-0.450311279515466,0.936707714991982,0.708380406186981,-0.468647287707221,0.354574063407955,-0.246634655717582,-0.00921237772707382,-0.595912405700819,-0.57568162226261,-0.113910176982092,-0.0246120063374677,0.196001952806192,0.0138016541409422,0.103758331023198,0.364297540595235,-0.382260574113217,0.092809187460487,0.0370505169810008,12.99]
input_data = [4462,-2.30334956758553,1.759247460267,-0.359744743330052,2.33024305053917,-0.821628328375422,-0.0757875706194599,0.562319782266954,-0.399146578487216,-0.238253367661746,-1.52541162656194,2.03291215755072,-6.56012429505962,0.0229373234890961,-1.47010153611197,-0.698826068579047,-2.28219382856251,-4.78183085597533,-2.61566494476124,-1.33444106667307,-0.430021867171611,-0.294166317554753,-0.932391057274991,0.172726295799422,-0.0873295379700724,-0.156114264651172,-0.542627889040196,0.0395659889264757,-0.153028796529788,239.93]
input_data_np = np.asarray(input_data).reshape(1,-1)
prediction = model.predict (input_data_np)

print(prediction)
print('Legit Transaction' if prediction[0] == 0 else 'Fraudulent Transaction')

[1.]
Fraudulent Transaction


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
